# Exploring OPTAA sensor data
In preparation for an OOIFB summer school in July 2023, I am developing a baseline knowledge about the spectrophotometer data that is available from OOI. The week-long summer school will educate OOI data users on how to access OPTAA (Spectrophotometer) instrument data, prepare it for analysis, and where the data comes from.

I am requesting and downloading data from one of the Coastal Pioneer NES array nodes that had an OPTAA instrument on a more recent deployment. After checking the content and structure of the downloaded dataset, I will plot a couple of different views of the data. I am interested in whether annotated data is already removed from the dataset, and how the visualizations I create will compare to what is currently available at [OOI Data Explorer](https:\\dataexplorer.oceanobservatories.org).

In [2]:
# Import modules needed for this notebook

import ooinet
import ooi_data_explorations as ooitools
import numpy
import pandas
import xarray
import dask
import matplotlib.pyplot as plt

# eventually when I have source code in this project folder, I will include the following line:
# import optaa_data_explorations as optaa

In [3]:
# Pick reference designator for instrument of interest
refdes = 'CP03ISSM-RID27-01-OPTAAD000'

[site, node, sensor] = refdes.split('-', 2)

In [ ]:
# Show available data recovery methods for refdes


In [ ]:
# Show available data streams for refdes and method

In [ ]:
# Show deployments for refdes (& get dates? How will this be used to build the data request?)